In [16]:
import os
import PyPDF2
import pandas as pd 
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Paths (scalable for 10+ reports)
REPORT_DIR = '..data/reports'
reports = {
    'McKinsey_State': 'McKinsey_State.pdf',
    'BCG_Reckoning': 'BCG_Reckoning.pdf',
    'KPMG_Insights': 'KPMG_Insights.pdf',
    'EY_FPA': 'EY_FPA.pdf',
    'McKinsey_Bank': 'McKinsey_Bank.pdf'
}

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
import os
print(os.listdir('../data/reports'))

['McKinsey_State.pdf', 'EY_FPA.pdf', 'BCG_Reckoning.pdf', 'KPMG_Insights.pdf', 'McKinsey_Bank.pdf']


In [20]:
import os
print(os.getcwd())

/workspaces/genai-finance-nlp-analysis/notebooks


In [17]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF. Why? PDFs have structured text for NLP."""
    try:
        text = ''
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + '\n'
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ''
    
extracted_texts = {}
for name, path in reports.items():
    full_path = os.path.join(REPORT_DIR, path)
    extracted_texts[name] = extract_text_from_pdf(full_path)
    print(f"Extracted {len(extracted_texts[name])} chars from {name}")


Error reading ..data/reports/McKinsey_State.pdf: [Errno 2] No such file or directory: '..data/reports/McKinsey_State.pdf'
Extracted 0 chars from McKinsey_State
Error reading ..data/reports/BCG_Reckoning.pdf: [Errno 2] No such file or directory: '..data/reports/BCG_Reckoning.pdf'
Extracted 0 chars from BCG_Reckoning
Error reading ..data/reports/KPMG_Insights.pdf: [Errno 2] No such file or directory: '..data/reports/KPMG_Insights.pdf'
Extracted 0 chars from KPMG_Insights
Error reading ..data/reports/EY_FPA.pdf: [Errno 2] No such file or directory: '..data/reports/EY_FPA.pdf'
Extracted 0 chars from EY_FPA
Error reading ..data/reports/McKinsey_Bank.pdf: [Errno 2] No such file or directory: '..data/reports/McKinsey_Bank.pdf'
Extracted 0 chars from McKinsey_Bank
